In [75]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support

In [76]:
ids = []
with open('/home/thomas/git/datascience/MSDataSci/505/data/project_three/ids.txt') as id_file:
    for line in id_file:
        line = line.replace('.txt\n','')
        ids.append(line)

iqs = []
iqs_labels = []
iqs_binary_labels = []
iqs_ternary_labels = []
matrices = []
vectors = []
data_dict = {}
i = 0
for id in ids:
    data_dict[id] = {}
    with open('/home/thomas/git/datascience/MSDataSci/505/data/project_three/dataset/{}.txt'.format(id)) as iq_file:
        iq = float(iq_file.readline().replace('\n',''))
        iqs.append(np.array([iq]))
        iqs_labels.append([iq,int(0 if iq<100 else 1),str('above average' if iq>110 else ('below average' if iq<90 else 'average'))])
        iqs_binary_labels.append(0 if iq<100 else 1)
        iqs_ternary_labels.append(2 if iq>110 else (0 if iq<90 else 1))
        data_dict[id]['iq'] = [iq,int(0 if iq<100 else 1),str('above average' if iq>110 else ('below average' if iq<90 else 'average'))]
    
    matrix = np.genfromtxt('/home/thomas/git/datascience/MSDataSci/505/data/project_three/dataset/{}.csv'.format(id),delimiter=',') 
    vector = np.array([np.array(np.genfromtxt('/home/thomas/git/datascience/MSDataSci/505/data/project_three/dataset/{}_vec.csv'.format(id),delimiter=','))])

    matrices.append(matrix)
    vectors.append(vector[0])

    data_dict[id]['pid'] = i
    data_dict[id]['matrix'] = matrix
    data_dict[id]['vector'] = vector

    i += 1
    
iqs_binary_labels = np.array(iqs_binary_labels)
    
matrices = np.array(matrices)
vects = np.array(vectors)
iqs = np.array(iqs)
iqs_binary_labels = np.array(iqs_binary_labels)
iqs_ternary_labels = np.array(iqs_ternary_labels)
regions = list(csv.reader(open("/home/thomas/git/datascience/MSDataSci/505/data/project_three/Atlas_regions.csv")))

In [77]:
# mean_log_dist = exp(mu+((sigma**2)/2))
pca = PCA(n_components=10)
pca.fit(vects)
vectors = pca.fit_transform(vectors)

In [78]:
bin_pipeline = Pipeline([('feature_selection',PCA()),
                     ('scaling',StandardScaler()),
                     ('classifier',LogisticRegression(multi_class='auto',solver='liblinear'))])

kf = KFold(n_splits=3,shuffle=True)
kf.get_n_splits(vectors)

for test_idx, train_idx in kf.split(vectors):
    vectors_train, vectors_test = vectors[train_idx], vectors[test_idx]
    iqs_binary_labels_train, iqs_binary_labels_test = iqs_binary_labels[train_idx], iqs_binary_labels[test_idx]

    bin_pipeline.fit(vectors_train,iqs_binary_labels_train)
    score = bin_pipeline.score(vectors_train,iqs_binary_labels_train)
    bin_scores.append(score)
    Yhat = pipeline.predict(vectors_test)
    metrics = precision_recall_fscore_support(iqs_binary_labels_test,Yhat)
    print('precision: {}'.format(str(metrics[0])))
    print('recall: {}'.format(str(metrics[1])))
    print('fscore: {}'.format(str(metrics[2])))
    print('support: {}'.format(str(metrics[3])))
    print('Fold Score: {0:0.2f}'.format(score))
    print('-----------------------')


precision: [0. 0. 0.]
recall: [0. 0. 0.]
fscore: [0. 0. 0.]
support: [ 8 30  0]
Fold Score: 0.95
-----------------------
precision: [0. 0. 0.]
recall: [0. 0. 0.]
fscore: [0. 0. 0.]
support: [ 7 32  0]
Fold Score: 1.00
-----------------------
precision: [0. 0. 0.]
recall: [0. 0. 0.]
fscore: [0. 0. 0.]
support: [ 7 32  0]
Fold Score: 0.95
-----------------------


/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [85]:
ter_pipeline = Pipeline([('feature_selection',PCA()),
                         ('scaling',StandardScaler()),
                         ('classifier',SVC())])

kf = KFold(n_splits=2,shuffle=True)
kf.get_n_splits(vectors)

for test_idx, train_idx in kf.split(vectors):
    vectors_train, vectors_test = vectors[train_idx], vectors[test_idx]
    iqs_ternary_labels_train, iqs_ternary_labels_test = iqs_ternary_labels[train_idx], iqs_ternary_labels[test_idx]

    ter_pipeline.fit(vectors_train,iqs_ternary_labels_train)
    score = ter_pipeline.score(vectors_train,iqs_ternary_labels_train)
    bin_scores.append(score)
    Yhat = pipeline.predict(vectors_test)
    metrics = precision_recall_fscore_support(iqs_ternary_labels_test,Yhat)
    print('precision: {}'.format(str(metrics[0])))
    print('recall: {}'.format(str(metrics[1])))
    print('fscore: {}'.format(str(metrics[2])))
    print('support: {}'.format(str(metrics[3])))
    print('Fold Score: {0:0.2f}'.format(score))

precision: [0.        0.        0.5862069]
recall: [0. 0. 1.]
fscore: [0.         0.         0.73913043]
support: [ 2 10 17]
Fold Score: 0.76
precision: [0.         0.         0.55172414]
recall: [0. 0. 1.]
fscore: [0.         0.         0.71111111]
support: [ 4  9 16]
Fold Score: 0.86


/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/thomas/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [80]:

lin_reg_scores = []
kf = KFold(n_splits=7,shuffle=True)
for train_index, test_index in kf.split(vectors,iqs):

    vectors_train, vectors_test = vectors[train_index], vectors[test_index]
    iqs_train, iqs_test = iqs[train_index], iqs[test_index]

    linreg = linear_model.LinearRegression()
    linreg.fit(vectors_train,iqs_train)
    Yhat = linreg.predict(vectors_test)

    # plt.hist(vectors[:,0],bins=10)
    # plt.hist(new_columns[8],bins=10)
    # plt.show()

    # plt.boxplot(vectors[:,0:50])

    # plt.boxplot(vectors[:,51:100])
    # plt.show()
    R_val = (pearsonr(iqs_test, Yhat)[0])
    MSE = mean_squared_error(iqs_test, Yhat)
    MAE = mean_absolute_error(iqs_test, Yhat)
    print('MSE: {0:0.3f}'.format(MSE))
    print('MAE: {0:0.3f}'.format(MAE))
    print('R: {0:0.3f}'.format(float(R_val)))
    print('Explained Variance: {0:0.3f}'.format(float(R_val)**2))
    print('================')
    
    lin_reg_scores.append(R_val[0]**2)
print('\nFold R-squared scores mean: {} and stdev: {}'.format(np.mean(lin_reg_scores),np.std(lin_reg_scores)))    

# plt.hist(scores,bins=6)
# plt.show()


MSE: 228.556
MAE: 12.190
R: -0.369
Explained Variance: 0.136
MSE: 235.721
MAE: 13.071
R: 0.182
Explained Variance: 0.033
MSE: 154.373
MAE: 9.742
R: 0.364
Explained Variance: 0.132
MSE: 270.153
MAE: 13.348
R: -0.387
Explained Variance: 0.150
MSE: 320.123
MAE: 16.010
R: 0.003
Explained Variance: 0.000
MSE: 128.637
MAE: 9.787
R: -0.419
Explained Variance: 0.175
MSE: 93.586
MAE: 7.947
R: -0.550
Explained Variance: 0.303

Fold R-squared scores mean: 0.1328103169754102 and stdev: 0.09141644546028044
